# MoodSync: Emotion-Responsive Lighting System

**MoodSync** is a unique project that changes the lighting in a room based on the detected emotion from a user's facial expression. The project uses a Convolutional Neural Network (CNN) to classify emotions from facial images and adjusts the brightness of Philips Hue lights accordingly.

## Project Story

The inspiration for MoodSync came from a simple moment. I was hanging out with my sister, and when I was leaving her room, she asked me to dim the lights because she was feeling down about a recent math test score. This got me thinking—what if there was a way for lights to adjust automatically based on emotions? MoodSync is the result of this thought, combining computer vision and IoT to create an emotion-responsive environment.


![FlowChart](FlowChart.png)


## Setup and Installation

To run MoodSync, you need to install the following libraries:
- **TensorFlow**: For building and training the CNN model.
- **Flask**: For creating the web app that hosts the model.
- **Requests**: To interact with the Philips Hue API.
- **Pillow**: For image processing.

In [ ]:
!pip install tensorflow flask requests pillow

In [ ]:
# TensorFlow and Keras for building the CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Flask for the web app
from flask import Flask, render_template, request, jsonify

# Requests for interacting with the Philips Hue API
import requests

# Pillow for image processing
from PIL import Image
import numpy as np

# Additional utilities
import os
import json


## Data Preparation

For this project, we use a facial emotion dataset. The dataset is organized by emotions (e.g., Happy, Sad, Neutral), and each category contains images of faces expressing that emotion.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths for training and validation data
training_dir = 'images/images/train'
validation_dir = 'images/images/validation'

# Image augmentation for training data
training_data = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40,
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation data
validation_data = ImageDataGenerator(rescale=1./255)

# Load images from directories
training_generator = training_data.flow_from_directory(
    training_dir, target_size=(64, 64), batch_size=64, class_mode='categorical'
)

validation_generator = validation_data.flow_from_directory(
    validation_dir, target_size=(64, 64), batch_size=64, class_mode='categorical'
)


## Model Building

MoodSync uses a Convolutional Neural Network (CNN) model to classify emotions based on facial images. The model architecture includes multiple convolutional and pooling layers to extract facial features, followed by dense layers for classification.


In [ ]:
from tensorflow.keras import layers, models

# Building the model
model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(64,64,3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2), padding='same'))

model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2), padding='same'))

model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2), padding='same'))

model.add(layers.Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2), padding='same'))

model.add(layers.Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2), padding='same'))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


## Model Training

The model is trained for 50 epochs with early stopping to prevent overfitting. This helps the model to learn from the dataset effectively and stop training when improvements stop. 


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set callbacks for early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('model_weights.h5', monitor='val_accuracy', save_best_only=True)

# Train the model
history = model.fit(
    training_generator, 
    epochs=50,
    validation_data=validation_generator, 
    callbacks=[early_stopping, checkpoint]
)


## Model Evaluation

After training, we evaluate the model's performance on the validation dataset to see how well it generalizes to unseen data.


In [ ]:
# Evaluate the model
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")


## Flask Application

The Flask application serves as the interface for MoodSync. Users upload images, and the application predicts their emotion using the CNN model. Based on the predicted emotion, the Philips Hue lights are adjusted to match the user's mood.


## Frontend Code (HTML, CSS, JavaScript)

The frontend includes a simple interface where users can capture an image, submit it to the Flask backend, and view the detected emotion. Below is the HTML, CSS, and JavaScript code for the interface.


## Conclusion

MoodSync is an innovative project that demonstrates the integration of computer vision with IoT to create an emotion-responsive environment. While this is just a proof-of-concept, it shows how technology can be used to create a more personalized and responsive atmosphere.

### Future Improvements
- Extend the range of detectable emotions.
- Integrate with additional smart home systems.
- Add options for different lighting effects.

Thank you for exploring MoodSync!
